## Introduction

In this lab, you'll practice your knowledge of one-to-many and many-to-many relationships!

## Objectives

You will be able to:

* Explain one-to-many and many-to-many joins as well as implications for the size of query results
* Query data using one-to-many and many-to-many joins

## One-to-Many and Many-to-Many Joins
<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png' width="600">

## Connect to the Database

Include the relevant imports, then connect to the database located at `data.sqlite`.

In [17]:
# Your code here
import sqlite3
import pandas as pd
conn = sqlite3.connect('data.sqlite')


In [18]:
cur = conn.cursor()


## Employees and Their Offices (a One-to-One Join)

Select all of the employees including their first name and last name along with the city and state of the office that they work out of (if they have one). Include all employees and order them by their first name, then their last name.

In [32]:
# Query to check the schema of employees table
cur.execute("PRAGMA table_info(employees)").fetchall()

# Query to check the schema of offices table
cur.execute("PRAGMA table_info(offices)").fetchall()


[(0, 'officeCode', 'INTEGER', 0, None, 0),
 (1, 'city', 'TEXT', 0, None, 0),
 (2, 'phone', 'TEXT', 0, None, 0),
 (3, 'addressLine1', 'TEXT', 0, None, 0),
 (4, 'addressLine2', 'TEXT', 0, None, 0),
 (5, 'state', 'TEXT', 0, None, 0),
 (6, 'country', 'TEXT', 0, None, 0),
 (7, 'postalCode', 'TEXT', 0, None, 0),
 (8, 'territory', 'TEXT', 0, None, 0)]

In [33]:
cur.execute("PRAGMA table_info(employees)").fetchall()


[(0, 'employeeNumber', 'INTEGER', 0, None, 0),
 (1, 'lastName', 'TEXT', 0, None, 0),
 (2, 'firstName', 'TEXT', 0, None, 0),
 (3, 'extension', 'TEXT', 0, None, 0),
 (4, 'email', 'TEXT', 0, None, 0),
 (5, 'officeCode', 'INTEGER', 0, None, 0),
 (6, 'reportsTo', 'INTEGER', 0, None, 0),
 (7, 'jobTitle', 'TEXT', 0, None, 0)]

In [34]:
# Your code here
q = """
    SELECT employees.firstName, employees.lastName, offices.city, offices.state
    FROM employees
    LEFT JOIN offices
    ON employees.officeCode = offices.officeCode
    ORDER BY employees.firstName, employees.lastName;
"""
df = pd.DataFrame(cur.execute(q).fetchall(), columns=["First Name", "Last Name", "City", "State"])
df

,First Name,Last Name,City,State
0,Andy,Fixter,Sydney,
1,Anthony,Bow,San Francisco,CA
2,Barry,Jones,London,
3,Diane,Murphy,San Francisco,CA
4,Foon Yue,Tseng,NYC,NY
5,George,Vanauf,NYC,NY
6,Gerard,Bondur,Paris,
7,Gerard,Hernandez,Paris,
8,Jeff,Firrelli,San Francisco,CA
9,Julie,Firrelli,Boston,MA


## Customers and Their Orders (a One-to-Many Join)

Select all of the customer contacts (first and last names) along with details for each of the customers' order numbers, order dates, and statuses.

In [35]:
# Your code here
q = """
    SELECT customers.contactFirstName, customers.contactLastName, orders.orderNumber, orders.orderDate, orders.status
    FROM customers
    LEFT JOIN orders
    ON customers.customerNumber = orders.customerNumber
    ORDER BY customers.contactFirstName, customers.contactLastName;
"""
df = pd.DataFrame(cur.execute(q).fetchall(), columns=["First Name", "Last Name", "Order Number", "Order Date", "Status"])
df

,First Name,Last Name,Order Number,Order Date,Status
0,Adrian,Huxley,10139.0,2003-07-16,Shipped
1,Adrian,Huxley,10270.0,2004-07-19,Shipped
2,Adrian,Huxley,10361.0,2004-12-17,Shipped
3,Adrian,Huxley,10420.0,2005-05-29,In Process
4,Akiko,Shimamura,10258.0,2004-06-15,Shipped
...,...,...,...,...,...
345,Yoshi,Tamuri,10206.0,2003-12-05,Shipped
346,Yoshi,Tamuri,10313.0,2004-10-22,Shipped
347,Yu,Choi,10242.0,2004-04-20,Shipped
348,Yu,Choi,10319.0,2004-11-03,Shipped


## Customers and Their Payments (Another One-to-Many Join)

Select all of the customer contacts (first and last names) along with details for each of the customers' payment amounts and date of payment. Sort these results in descending order by the payment amount. 

In [36]:
# Your code here
q = """
    SELECT customers.contactFirstName, customers.contactLastName, payments.amount, payments.paymentDate
    FROM customers
    LEFT JOIN payments
    ON customers.customerNumber = payments.customerNumber
    ORDER BY payments.amount DESC;
"""
df = pd.DataFrame(cur.execute(q).fetchall(), columns=["First Name", "Last Name", "Payment Amount", "Payment Date"])
df


,First Name,Last Name,Payment Amount,Payment Date
0,Diego,Freyre,120166.58,2005-03-18
1,Diego,Freyre,116208.40,2004-12-31
2,Susan,Nelson,111654.40,2003-08-15
3,Eric,Natividad,105743.00,2003-12-26
4,Susan,Nelson,101244.59,2005-03-05
...,...,...,...,...
292,Sven,Ottlieb,NaN,None
293,Carmen,Anton,NaN,None
294,Hanna,Moos,NaN,None
295,Alexander,Semenov,NaN,None


## Orders, Order Details, and Product Details (a Many-to-Many Join)

Select all of the customer contacts (first and last names) along with the product names, quantities, and date ordered for each of the customers and each of their orders. Sort these in descending order by the order date.

> Note: This will require joining 4 tables! This can be tricky! Give it a shot, and if you're still stuck, turn to the next section where you'll see how to write subqueries that can make complex queries such as this much simpler!

In [37]:
# Your code here
q = """
    SELECT customers.contactFirstName, customers.contactLastName, products.productName, orderdetails.quantityOrdered, orders.orderDate
    FROM customers
    LEFT JOIN orders
    ON customers.customerNumber = orders.customerNumber
    LEFT JOIN orderdetails
    ON orders.orderNumber = orderdetails.orderNumber
    LEFT JOIN products
    ON orderdetails.productCode = products.productCode
    ORDER BY orders.orderDate DESC;
"""
df = pd.DataFrame(cur.execute(q).fetchall(), columns=["First Name", "Last Name", "Product Name", "Quantity Ordered", "Order Date"])
df


,First Name,Last Name,Product Name,Quantity Ordered,Order Date
0,Janine,Labrune,1962 LanciaA Delta 16V,38.0,2005-05-31
1,Janine,Labrune,1957 Chevy Pickup,33.0,2005-05-31
2,Janine,Labrune,1998 Chrysler Plymouth Prowler,28.0,2005-05-31
3,Janine,Labrune,1964 Mercedes Tour Bus,38.0,2005-05-31
4,Janine,Labrune,1926 Ford Fire Engine,19.0,2005-05-31
...,...,...,...,...,...
3015,Sven,Ottlieb,None,NaN,None
3016,Carmen,Anton,None,NaN,None
3017,Hanna,Moos,None,NaN,None
3018,Alexander,Semenov,None,NaN,None


## Summary

In this lab, you practiced your knowledge of one-to-many and many-to-many relationships!